## Day 3

### Part 1
https://adventofcode.com/2024/day/3


So what is going on here?
* mul(X,Y) multiplies X and Y where X, Y are three digit numbers (with leading zeroes, possibly)
* but there are some invalid commands too. These should do nothing.
* find the sum of every mul operation and return.

### Approach
sliding window problem
* have a start index and an end index
* move the start index until we find an 'm'; then, lock the start index
* start advincing the end index, validating each character until we get a fully formed mul instruction. If at any point validation fails, then set start index to end and continue moving.




In [30]:
// Checks if a report is safe (returns true) or unsafe (returns false)
using System.Text.RegularExpressions;
int get_mul_sum(string input){
    
    // use regex
    string pattern = @"mul\(([0-9]{1,3}),([0-9]{1,3})\)";
    Regex regex = new Regex(pattern);

    var matches = regex.Matches(input);
    
    int mul_sum = 0;

    foreach(Match match in matches){
        // we only want the number parts here
        mul_sum += int.Parse(match.Groups[1].Value) * int.Parse(match.Groups[2].Value);
    }

    return mul_sum;
}

In [27]:
string testInput1 = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))";
int result1 = get_mul_sum(testInput1);
Console.WriteLine(result1);

System.Text.RegularExpressions.MatchCollection
161


## File Reading

* Read in the file one line at a time. 
* Break that line into a list of numbers, separated on whitespace.

In [31]:
using System.IO;
int result = 0;
foreach (string line in File.ReadLines("day3-input.txt")){
    // int p1 = Convert.ToInt32(line.Substring(0, 5));
    // int p2 = Convert.ToInt32(line.Substring(8, 5));
    
    result += get_mul_sum(line);
    
}

Console.WriteLine(result);

182619815


## Part 2

We change what qualifies as a bad report: We can now tolerate a single bad level.

What this means:
* any report where removing a single level would make it safe also counts as safe.

How to handle the whole dampening thing:
* use a 0 and 1 index tracker
* when working normally, advance both
* when encountering the first failure, increment 1 tracker but not 0.
* when encountering the second failure, return false as before.

Current issue:
* getting the wrong number. I think it's because i always try to skip the deep number when i find an issue, but really i could also skip the deep number too.
* try a list method, i guess. Use recursion with both cut lists and return an OR.

Still not working
* next thing would be to try it exhaustively, removing every value.
* O(n^2)

In [22]:
// Checks if a report is safe (returns true) or unsafe (returns false)
// Has dampening, which means that any report where removing a single level would make it safe also counts as safe.
bool Validate_Report_WithDampening(List<int> report, bool canTolerate){
    if (report.Count == 1) return true;

    bool ascending = true;
    if (report[0] > report[1]) ascending = false;

    int deep = 1;
    int shallow = 0;
    while (deep < report.Count){
        int dif = report[deep] - report[shallow];
        // Console.WriteLine(report[shallow]);
        // Console.WriteLine(report[deep]);
        // Console.WriteLine(dif);

        // when ascending, [i] > [i-1], i.e. dif must be positive
        // when descending, [i] < [i-1], i.e. dif must be negative
        if ((ascending && dif > 0 && dif < 4) || (!ascending && dif < 0 && dif > -4)){
            shallow += 1;
            deep += 1;
        }
        else if (canTolerate){
            canTolerate = false;
            // try both subsections of the list and return their OR.
            List<int> cutShallow = new List<int>(report);
            List<int> cutDeep = new List<int>(report);

            if (deep + 1 < report.Count) cutShallow.RemoveAt(deep + 1);
            cutDeep.RemoveAt(shallow);

            return Validate_Report_WithDampening(cutShallow, canTolerate) 
                || Validate_Report_WithDampening(cutDeep, canTolerate);

        }
        else return false;
    }
    return true;
}

In [20]:
List<int> testList1 = new List<int>(){7,6,4,2,1};
List<int> testList2 = new List<int>(){1,2,7,8,9};
List<int> testList3 = new List<int>(){9,7,6,2,1};
List<int> testList4 = new List<int>(){1,3,2,4,5};
List<int> testList5 = new List<int>(){8,6,4,4,1};
List<int> testList6 = new List<int>(){1,3,6,7,9};

bool result1 = Validate_Report_WithDampening(testList1, true);
bool result2 = Validate_Report_WithDampening(testList2, true);
bool result3 = Validate_Report_WithDampening(testList3, true);
bool result4 = Validate_Report_WithDampening(testList4, true);
bool result5 = Validate_Report_WithDampening(testList5, true);
bool result6 = Validate_Report_WithDampening(testList6, true);

Console.WriteLine(result1);
Console.WriteLine(result2);
Console.WriteLine(result3);
Console.WriteLine(result4);
Console.WriteLine(result5);
Console.WriteLine(result6);

True
False
False
True
True
True


In [21]:
using System.IO;
int safeSum = 0;
foreach (string line in File.ReadLines("day2-input.txt")){
    // int p1 = Convert.ToInt32(line.Substring(0, 5));
    // int p2 = Convert.ToInt32(line.Substring(8, 5));
    
    // split line to report.
    string[] reportStrs = line.Split(' ');
    // Console.WriteLine(reportStrs);

    // convert strings to ints.
    int[] reportInts = Array.ConvertAll(reportStrs, int.Parse);
    List<int> reportIntsList = reportInts.ToList();
    // Console.WriteLine(reportInts);
    // Console.WriteLine(reportIntsList);
    
    // Check safety and increase safeSum.
    if (Validate_Report_WithDampening(reportIntsList, true)) safeSum += 1;
}

Console.WriteLine((safeSum));

587
